In [ ]:

# from google.colab import drive
# drive.mount('/content/drive')

!unzip /content/audio_abstract_webapp.zip

Archive:  /content/audio_abstract_webapp.zip
   creating: static/
   creating: static/css/
  inflating: static/css/gcc.css      
   creating: templates/
  inflating: templates/index.html    
  inflating: templates/next.html     
  inflating: transcr.ipt.txt         


In [ ]:
%cd Audio_webapp/

[Errno 2] No such file or directory: 'Audio_webapp/'
/content


In [1]:

!pip install pyngrok
!pip install -q git+https://github.com/openai/whisper.git > /dev/null
!pip install -q git+https://github.com/pyannote/pyannote-audio > /dev/null

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 718.7/718.7 kB 10.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-7.0.0-py3-none-any.whl size=21129 sha256=0c9984f7cdaf8e46b3579d2f7c5017e755b199b9fa71b628dc40cedd269733c1
  Stored in directory: /root/.cache/pip/wheels/60/29/7b/f64332aa7e5e88fbd56d4002185ae22dcdc83b35b3d1c2cbf5
Successfully built pyngrok


In [ ]:
!pip install sentencepiece
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 109.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 67.0 MB/s eta 0:00:00


In [ ]:
num_speakers = 2 #@param {type:"integer"}

language = 'English' #@param ['any', 'English']

model_size = 'medium' #@param ['tiny', 'base', 'small', 'medium', 'large']


model_name = model_size
if language == 'English' and model_size != 'large':
  model_name += '.en'

In [ ]:


import whisper
import datetime

import subprocess

import torch
import pyannote.audio
from pyannote.audio.pipelines.speaker_verification import PretrainedSpeakerEmbedding
embedding_model = PretrainedSpeakerEmbedding(
    "speechbrain/spkrec-ecapa-voxceleb",
    device=torch.device("cuda"))

from pyannote.audio import Audio
from pyannote.core import Segment

import wave
import contextlib

from sklearn.cluster import AgglomerativeClustering
import numpy as np

In [ ]:
import json
import pickle
import sentencepiece
import pandas as pd
import numpy as np
import transformers
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM

In [ ]:
# import inspect
# import shutil
# module_path = inspect.getfile(pyannote.audio)
# destination_path = os.path.join(os.getcwd(), "pyannote.audio.py")
# shutil.copyfile(module_path,destination_path)

In [ ]:
model= whisper.load_model(model_size)


100%|██████████████████████████████████████| 1.42G/1.42G [00:13<00:00, 113MiB/s]


In [ ]:
import sentencepiece

model2 = AutoModelForSeq2SeqLM.from_pretrained("google/pegasus-cnn_dailymail")

# model.save_pretrained("my_model_checkpoint")
tokenizer = AutoTokenizer.from_pretrained("google/pegasus-cnn_dailymail")

In [ ]:
def generate_summary(text,ma, min):
    summarizer = pipeline("summarization", model=model2, tokenizer=tokenizer)
    summary = summarizer(text, max_length=ma,min_length=min, do_sample=True)

    return summary[0]["summary_text"]




In [ ]:
from flask import Flask, render_template, request
from pyngrok import ngrok
from werkzeug.utils import secure_filename
import contextlib

app = Flask(__name__)
ngrok.set_auth_token('2PrZXp4F1mikZ3DyQYsXf9TEgOm_YBoVXLga1wW4vfAWb6SG')
public_url = ngrok.connect(5000).public_url
print(f"Ngrok URL: {public_url}")


@app.route('/', methods=['GET', 'POST'])
def index():


    if request.method == 'POST':

        audio_file = request.files['audio']
        audio_file.save(audio_file.filename)

        path= secure_filename(audio_file.filename)


        # Process the audio file here

        if path[-3:] != 'wav':
              re=subprocess.call(['ffmpeg', '-i', path, 'audio.wav', '-y'])
              path='audio.wav'


        model = whisper.load_model(model_size)
        result = model.transcribe(path)
        segments = result["segments"]
        with contextlib.closing(wave.open(path,'r')) as f:
            frames = f.getnframes()
            rate = f.getframerate()
            duration = frames / float(rate)


        audio = Audio()

        def segment_embedding(segment):
            start = segment["start"]
            # Whisper overshoots the end timestamp in the last segment
            end = min(duration, segment["end"])
            clip = Segment(start, end)
            waveform, sample_rate = audio.crop(path, clip)
            return embedding_model(waveform[None])


        embeddings = np.zeros(shape=(len(segments), 192))
        for i, segment in enumerate(segments):
            embeddings[i] = segment_embedding(segment)

        embeddings = np.nan_to_num(embeddings)
        clustering = AgglomerativeClustering(num_speakers).fit(embeddings)
        labels = clustering.labels_
        for i in range(len(segments)):
            segments[i]["speaker"] = 'SPEAKER ' + str(labels[i] + 1)

        def time(secs):
            return datetime.timedelta(seconds=round(secs))

        f = open("transcript.txt", "w")

        for (i, segment) in enumerate(segments):
            if i == 0 or segments[i - 1]["speaker"] != segment["speaker"]:
              f.write("\n" + segment["speaker"] + ':' )
            f.write(segment["text"][1:] + ' '+'\n')
        f.close()
        input_text=open('transcript.txt','r').read()
        word_count = len(input_text.split(' '))
        abstract=generate_summary(input_text,int(word_count*0.5),int(word_count*0.2))
        return render_template('next.html',text=abstract)
        # return f"File {dia_text} has been uploaded and processed successfully!"



    else:
        return render_template('index.html')



if __name__ == '__main__':
    app.run()


Exception in thread Thread-17 (_monitor_process):
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.10/dist-packages/pyngrok/process.py", line 146, in _monitor_process
    self._log_line(self.proc.stdout.readline())
  File "/usr/lib/python3.10/encodings/ascii.py", line 26, in decode
    return codecs.ascii_decode(input, self.errors)[0]
UnicodeDecodeError: 'ascii' codec can't decode byte 0xc2 in position 184: ordinal not in range(128)


Ngrok URL: https://1e03-35-197-135-102.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [26/May/2023 04:45:51] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/May/2023 04:45:52] "GET /static/css/gcc.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/May/2023 04:45:53] "GET /favicon.ico HTTP/1.1" 404 -


7